In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from jupyter_plotly_dash import JupyterDash


#CRUD Python module file name and class name
from script import AnimalShelter





###########################
# Data Manipulation / Model
###########################
#your username and password and CRUD Python module name
username = "aacuser"
password = "M0nkeys11"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Animal Shelter')

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
#encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'html.Img(src=data:image/png;base64,{}.format(image_filename))'}),
    html.Center(html.B(html.H1('Rebekah Collett'))),
    html.Hr(),
    html.Div(dcc.RadioItems( id='filter-type', 
                            options=[ {'label': 'Water Rescue', 'value': 1}, 
                                      {'label': 'Mountain or Wilderness Rescue', 'value': 2},
                                      {'label': 'Disaster or Individual Tracking', 'value': 3},
                                      {'label': 'Reset', 'value': 4}],
                            value='0', labelStyle={'display': 'inline-block'} )),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    html.Hr(),
    dt.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
    # the features for your interactive data table to make it user-friendly for your client
    
        editable = True,
        filter_action = 'native',
        sort_action = 'native',
        sort_mode = 'multi',
        column_selectable = 'single',
        row_selectable = 'multi',
        row_deletable = True,
        selected_columns = [],
        selected_rows = [0],
        page_action = 'native',
        page_current = 0,
        page_size = 10,   
    
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-interactivity','data'),
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
### code to filter interactive data table with MongoDB queries

        df = pd.DataFrame.from_records(shelter.read({}))
        data=df.to_dict('records')
        
                        
        return data
         
@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
        } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
    
    
def update_graphs(viewData):
    #code for chart of your choice (e.g. pie chart)
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, values=df.age_upon_outcome_in_weeks, names=df.breed, hole=.3)    
        )]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data"),
     Input('datatable-interactivity', "derived_viewport_selected_rows")])
def update_map(viewData, row):
#code for your geolocation chart
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.loc[row]
    
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(
                    position=[
                        dff.loc[0,'location_lat'],
                        dff.loc[0,'location_long']
                    ],
                    children=[
                        dl.Tooltip(dff.loc['breed']),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.loc[0,'name']),
                        ])
                    ])
            ])
    ]

app